In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import nltk  
import numpy as np  
import random  
import string
import re

In [2]:
df = pd.DataFrame()
df  = pd.read_csv('labels_imperium_all.csv',encoding='utf-8',header=None)
labels = df[0]
df.head()

,0
0,1
1,0
2,0
3,0
4,0


In [3]:
df = pd.DataFrame()
df  = pd.read_csv('imperium_txts_all.csv',encoding='utf-8',header=None)
df.head()

,0
0,"""You fuck your dad."""
1,"""i really don't understand your point.\xa0 It ..."
2,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,"""listen if you dont wanna get married to a man..."
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [4]:
documents = df[0]
print(documents[0])
# lower case
documents = documents.str.lower() 
# remove punctuation
documents = documents.str.replace('[^\w\s]','') 
documents = documents.values.tolist()
for i in range(len(documents )):
    # tokenize
    documents [i] = nltk.word_tokenize(documents [i])
    # remove blanck spaces
    documents [i] = [re.sub(r'\W',' ',word) for word in documents [i] ]
print("Ther are :",len(documents)," 0:", documents[0])

"You fuck your dad."
Ther are : 8829  0: ['you', 'fuck', 'your', 'dad']


Cleaning the data: Remove puntuation

In [210]:
wordfreq = {}
for sentence in documents:
    for token in sentence:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
print("Ther are :",len(wordfreq)," unique words")

Ther are : 30675  unique words


The wordfreq save each unique word and the number of time that his words appears

In [287]:
# Use vectorize function of numpy 
length_checker = np.vectorize(len) 
  
# Find the length of each element 
documents_length = length_checker(documents) 
  
# Print the length of each element 
print(documents_length) 

[  4  15  64 ...   8  52 188]


In [223]:
# List of keys to be deleted from dictionary
selectedKeys = list() 
 
# Iterate over the dict and put to be deleted keys in the list
for (key, value) in wordfreq.items():
    if value < 5:
        selectedKeys.append(key)
    if len(key) < 3:
        selectedKeys.append(key)
    if len(key) > 30:
        selectedKeys.append(key)
 
# Iterate over the list and delete corresponding key from dictionary
for key in selectedKeys:
    if key in wordfreq:
        del wordfreq[key]
print("Ther are :",len(wordfreq)," unique words")

Ther are : 4110  unique words


In [228]:
# Saving each unique word index
word_idx = {}
idx = 0
for token in wordfreq:
    word_idx[token] =  idx
    idx += 1
print(word_idx['florida'])

3020


In [8]:
sentence_vectors = []
for sentence in documents:
    sent_vec = []
    for token in wordfreq:
        if token in sentence:
            df = sentence.count(token)
            sent_vec.append(df)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [9]:
sentence_vectors = np.asarray(sentence_vectors)
print("Matrix of dimension: (",sentence_vectors.shape[0],",",sentence_vectors.shape[1],")")

Matrix of dimension: ( 8829 , 4110 )


In [222]:
# Identifying the document with more terms
maxlen_doc = []
for doc in documents:
    maxlen_doc.append(len(doc))
print("Maximum document longitud: ",max(maxlen_doc))  
print(max(documents_length))
maxlen_doc = (max(documents_length))

Maximum document longitud:  2299
2299


Creating an document / index matrix

In [238]:
# We need to separete this in sTrain and Stest, we need to be careful to mantein the index in the training fase
num_docs = sentence_vectors.shape[0]
maximum = []
idx_matriz = np.zeros((num_docs, maxlen_doc)) # idx_matriz represents "stest" in Matlab
row = 0
for doc in documents:
    # If tern not in dicc, add '0'
    doc_inx = [ word_idx[x] if x in word_idx else 0 for x in doc] 
    doc_inx.extend(repeat(0, maxlen_doc - len(doc_inx)))  # Here we are padding, but it is NOT necessary
    idx_matriz[row] = (doc_inx)
    row += 1
    # Like this ingnore those words tht are not in dic : doc_inx = [wordfreq[x] for x in doc if x in wordfreq]
    maximum.append(len(doc_inx))
print("Maximum document longitud: ",max(maximum))   

Maximum document longitud:  2299


In [10]:
np.savetxt("document_matrix.csv", sentence_vectors, delimiter=",")

In [11]:
df  = pd.read_csv('document_matrix.csv',encoding='utf-8',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4100,4101,4102,4103,4104,4105,4106,4107,4108,4109
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [263]:
X_train = sentence_vectors[:6182]
X_test  = sentence_vectors[6182:]
Y_train = labels[:6182]
Y_test  = labels[6182:]
S_train = idx_matriz[:6182]
S_test  = idx_matriz[6182:]    
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(S_train.shape)
print(S_test.shape)

(6182, 4110)
(6182,)
(2647, 4110)
(2647,)
(6182, 2299)
(2647, 2299)


In [277]:
def csa(X,Y,X_test):
    num_terms = X.shape[1]
    num_class = 2
    num_docs = X.shape[0]

    TR = np.zeros((num_terms, num_class))
    # Term Representation    
    for j in range (0,num_class): # j = [0;1]
        docs = np.where(Y == j)[0] 
        for i in range(0,num_terms):
            tf_v = X[docs,i]
            TR[i][j] += sum(np.log2(1 + tf_v / documents_length [docs])) #this docleng is only exple until train size
    print("PRB: (",TR.shape[0],",",TR.shape[1],")")
    
    # Normalization 1' Sum all the weights vectors 
    n1 = np.sum(TR, axis = 0) #Sum by colum
    Tik = TR / n1

    # Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
    for i in range(0,num_terms):
        Tik[i] = Tik[i] / sum(Tik[i]) # Tik[i] = Tik[i] / sum(TR[i])

    print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
    
    # Document Representation for Xtrain
    DR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X[i] != 0)[0] # extrac idxs =! 0
        a = X[i][termos] / documents_length[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DR[i] = sum(np.multiply(a,Tik[termos]))
    print(DR.shape)
    
    # Document Representation for Xtest
    num_docs = X_test.shape[0]
    DR_T = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X_test[i] != 0)[0] # extrac idxs =! 0
        a = X_test[i][termos] / documents_length[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DR_T[i] = sum(np.multiply(a,Tik[termos]))
    print(DR_T.shape)
    return DR,DR_T


In [291]:
DRS,DR_TS = csa(X_train,Y_train,X_test)
print(DRS.shape)
print(DR_TS.shape)

PRB: ( 4110 , 2 )
Tik: ( 4110 , 2 )


C:\Users\DanielaFe7\Anaconda3\envs\RendesNeurais\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


(6182, 2)
(2647, 2)
(6182, 2)
(2647, 2)


In [302]:
for i in DR_TS:
    print(i)

[33.83341924 15.91658076]
[0.68726303 0.4460703 ]
[0.12087854 0.11349646]
[0.21131416 0.1954655 ]
[0.21009374 0.15354262]
[0.09731733 0.06934934]
[0.43758483 0.06241517]
[0.29786605 0.31324506]
[1.17303332 1.07696668]
[0.48164697 0.26835303]
[2.04237398 1.38619745]
[0.77750068 0.42249932]
[0.62225427 0.37774573]
[0.16153799 0.08846201]
[0.22256366 0.27743634]
[0.12713339 0.12286661]
[0.34517558 0.32149108]
[0.29078227 0.23553352]
[1.09166436 0.90833564]
[0.14160582 0.0806164 ]
[3.52136145 2.90720998]
[19.70467606 14.29532394]
[0.24617003 0.25382997]
[0.06920721 0.05579279]
[0.01506638 0.01667966]
[0.3459708  0.19249074]
[1.38050799 1.20772731]
[0.99719063 1.11392048]
[0.7614906  0.46073162]
[0.09881228 0.14031816]
[0.02916297 0.0512968 ]
[0.86874459 0.55230805]
[0.04574934 0.05537426]
[0.0975602 0.1024398]
[0.20274967 0.13058366]
[0.43283517 0.46716483]
[0.0616286 0.1383714]
[0.01834036 0.02817127]
[2.865483 2.334517]
[0.30795626 0.32840738]
[4.18940193 4.31059807]
[0.6127387 0.3872613

[1.60727833 0.96415024]
[0.37487352 0.18326602]
[0. 0.]
[0.13438357 0.07990214]
[1.2341074  1.05160689]
[0.06851903 0.23917328]
[0.02428159 0.04888915]
[0.36935552 0.26700812]
[0.07384069 0.10797749]
[0.2452005  0.19597597]
[0.17969183 0.39173674]
[0.37033535 0.26602828]
[0.24639989 0.20814557]
[0.30098913 0.19901087]
[0.78760633 0.54572701]
[0.15637138 0.17696195]
[0.24394527 0.15188806]
[2.20642945 1.88447964]
[0.17059547 0.25248145]
[0.27802669 0.30892983]
[0.11358761 0.07159758]
[0.4254881  0.35228968]
[0.08458413 0.06926203]
[0.13644269 0.0930655 ]
[1.71055147 0.95611519]
[0.26766954 0.28788602]
[5.49305423 4.07837434]
[2.95189135 2.71477531]
[0.26606488 0.27052049]
[0.02985159 0.021619  ]
[0.1169718  0.06821338]
[nan nan]
[0.42664871 0.39835129]
[0.35259275 0.44740725]
[0.86886951 0.49779715]
[0.10865261 0.09968072]
[0.4910638 0.4089362]
[0.06426163 0.04162072]
[0.0506686  0.02179517]
[0.8858447 0.9141553]
[4.86828164 4.00671836]
[10.64746768  8.01919899]
[0.63814931 0.86185069]


[0.22712855 0.32287145]
[0.16369595 0.15209353]
[0.49489118 0.33844215]
[0.62727598 0.7256652 ]
[0.25562393 0.16625107]
[0.9479425  0.82983528]
[1.25195411 0.92986407]
[0.21844772 0.14693689]
[0.20930591 0.12402742]
[0.10921224 0.09533321]
[0.17854415 0.1676097 ]
[0.33858887 0.44712542]
[1.98350809 1.39884485]
[0.17617396 0.25239747]
[0.42721531 0.28111802]
[0.04493209 0.045977  ]
[0.46893021 0.23106979]
[0.09407325 0.11645307]
[0.18077197 0.13172803]
[0.26084619 0.29470937]
[0.05727168 0.02606165]
[0.35663251 0.29042632]
[1.39279206 1.4643508 ]
[0.07168823 0.06164511]
[0.07285554 0.08168991]
[0.08219688 0.1905304 ]
[0.65346505 0.77510637]
[0.23003734 0.20329599]
[0.37817315 0.35516018]
[0.70361515 0.54638485]
[0.21111042 0.21746101]
[0.22123309 0.13590977]
[0.55972155 0.44027845]
[4.80403522 3.44596478]
[0.50890936 0.59109064]
[0.19175365 0.22491301]
[0.7600594  0.57327393]
[0.23155142 0.13208495]
[0.32464961 0.23417391]
[0.53297866 0.46702134]
[0.04725622 0.0397003 ]
[1.43964178 0.89

[0.02819473 0.01658139]
[0.44707283 0.43528011]
[0.7779333  0.75147846]
[0.04355222 0.03052185]
[0.65348189 0.34651811]
[1.53226554 1.26773446]
[0.17317547 0.12682453]
[0.42455753 0.20044247]
[0.0443955  0.02877523]
[0.27377631 0.28872369]
[0.02538777 0.01398231]
[1.3260149 1.4239851]
[0.13500661 0.06499339]
[0.35795349 0.27362546]
[0.09406157 0.07145567]
[0.24648967 0.1171467 ]
[0.19543424 0.20456576]
[nan nan]
[0.42452139 0.32547861]
[0.96935768 1.16397565]
[0.97821719 0.39678281]
[0.44718649 0.19567066]
[0.2889924 0.2110076]
[1.18531658 1.21468342]
[0.05836268 0.05852044]
[2.45039845 1.84960155]
[0.54057234 0.34178061]
[1.20661763 0.73088237]
[3.21516904 2.45149763]
[0.0846765  0.05045864]
[1.68459523 1.31540477]
[0.19359959 0.13973374]
[0.82211765 0.51121568]
[0.7243363 0.6356637]
[0.13501341 0.08720881]
[0.19055728 0.14277605]
[0.57002981 0.42997019]
[0.2332854  0.15004794]
[0.72784607 0.48268025]
[2.14387972 1.45612028]
[0.47474828 0.77525172]
[0.67328997 0.55747926]
[0.08185887 

[0.63135598 0.59086625]
[0.12886577 0.11355847]
[0.0488748  0.01779187]
[0.27312738 0.16165523]
[3.63396735 2.92158821]
[2.59012742 2.40987258]
[0.11350753 0.21982581]
[0.45304483 0.8802885 ]
[0.0546863 0.0362228]
[0.87730043 1.12269957]
[0.30551651 0.21448349]
[0.4524646  0.33700909]
[0.14036568 0.04481951]
[1.2321802  2.10115314]
[0.52178476 0.30174465]
[0.84945472 0.56304528]
[0.48897316 0.51102684]
[0.03971244 0.05552566]
[0.31779475 0.39649096]
[0.26136345 0.23863655]
[0.24057317 0.15073118]
[1.54578952 1.08578943]
[10.01108572  8.65558095]
[0.03496742 0.03775985]
[2.61707057 2.13292943]
[0.04549467 0.02997702]
[0.61755013 0.53629603]
[0.0845213  0.07337344]
[3.31264715 3.5444957 ]
[0.13344548 0.10032075]
[0.10076417 0.04209297]
[0.21454483 0.13429238]
[0.15989492 0.22472046]
[0.28073505 0.21926495]
[0.08618592 0.10429028]
[0.18083313 0.41916687]
[0.53009921 0.46990079]
[0.03048916 0.1810493 ]
[0.5558764 0.9441236]
[11.43026132  6.56973868]
[0.30444679 0.19555321]
[0.57297263 0.42

[2.83100224 1.74042633]
[2.09822727 1.0684394 ]
[1.37753406 1.12246594]
[3.11939939 1.88060061]
[0.11144626 0.31712516]
[1.31224579 0.74657774]
[0.95651097 0.58894358]
[0.42921281 0.23745386]
[1.26574161 0.90092506]
[31.84255523 24.15744477]
[nan nan]
[1.57043998 1.22956002]
[0.09202559 0.05431587]
[0.34682489 0.542064  ]
[0.30126971 1.03206363]
[0.54922654 0.78410679]
[1.14537986 1.23923552]
[2.84112827 2.04776062]
[0.45106722 0.45518278]
[3.911566 3.088434]
[1.19111488 0.53615785]
[0.07660161 0.07491354]
[1.0219535  0.52350105]
[0.09532915 0.10979906]
[0.12230994 0.12769006]
[0.95361968 0.71304698]
[0.20948885 0.29051115]
[0.80575768 1.19424232]
[0.56179776 0.43820224]
[0.32286922 0.17713078]
[3.76941632 2.98058368]
[0.60131366 0.39868634]
[0.25700719 0.24299281]
[0. 0.]
[0.76727961 1.39938706]
[0.03726359 0.08773641]
[0.91311708 0.5683644 ]
[1.12548257 0.71451743]
[0.71790349 0.28209651]
[1.1228009  0.69298857]
[0.22446869 0.23007677]
[1.82695347 1.67304653]
[1.05334452 0.94665548]


In [288]:
len(documents_length)

8829

PBR: u did this for all X, u shoul do this 4 xtrain and x test

In [281]:
num_terms = sentence_vectors.shape[1]
num_class = 2
TR = np.zeros((num_terms, num_class))
num_docs = sentence_vectors.shape[0]

In [292]:
for j in range (0,num_class): # j = [0;1]
        docs = np.where(labels == j)[0] 
        for i in range(0,num_terms):
            tf_v = sentence_vectors[docs,i]
            TR[i][j] += sum(np.log2(1 + tf_v / documents_length [docs]))

print("PRB: (",TR.shape[0],",",TR.shape[1],")")

# Normalization 1' Sum all the weights vectors 
n1 = np.sum(TR, axis = 0) #Sum by colum
Tik = TR / n1

# Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
for i in range(0,num_terms):
    Tik[i] = Tik[i] / sum(Tik[i]) # Tik[i] = Tik[i] / sum(TR[i])

print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")

PRB: ( 4110 , 2 )
Tik: ( 4110 , 2 )


In [293]:
# Document Representation
DR = np.zeros((num_docs, num_class))
for i in range(0,num_docs):
    termos = np.where(sentence_vectors[i] != 0)[0] # extrac idxs =! 0
    a = sentence_vectors[i][termos] / documents_length[i]# acces idxs content / len
    a = np.expand_dims(a, axis=1)
    DR[i] = sum(np.multiply(a,Tik[termos]))
DR.shape
# Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"
# none of the owrds bellow to dicc

(8829, 2)

In [299]:
np.equal(X_train,DRS)

array([[False, False],
       [False, False],
       [False, False],
       ...,
       [False, False],
       [False, False],
       [False, False]])

In [295]:
X_train = DR[:6182]
X_test  = DR[6182:]
Y_train = labels[:6182]
Y_test  = labels[6182:]
S_train = idx_matriz[:6182]
S_test  = idx_matriz[6182:]
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(S_train.shape)
print(S_test.shape)

(6182, 2)
(6182,)
(2647, 2)
(2647,)
(6182, 2299)
(2647, 2299)


In [179]:
#X_train, X_test, Y_train, Y_test = train_test_split( DR, labels, test_size=0.3, random_state=42)
#print(X_train.shape)
#print(Y_train.shape)
#print(X_test.shape)
#print(Y_test.shape)

In [17]:
from keras.models import Sequential
from keras.layers import Dense
import keras
#create model
model = Sequential()
#get number of columns in training data
#add model layers
model.add(Dense(10, activation='relu', input_shape=(num_class,)))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Using TensorFlow backend.





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
opt = keras.optimizers.SGD(decay=1e-14)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
model.fit(X_train, Y_train, epochs=100, batch_size=10, validation_data=(X_test, Y_test))


Train on 6180 samples, validate on 2649 samples
Epoch 1/100
6180/6180 [==============================] - 2s 317us/step - loss: 0.6268 - acc: 0.6662 - val_loss: 0.6007 - val_acc: 0.6844
Epoch 2/100
6180/6180 [==============================] - 1s 149us/step - loss: 0.5977 - acc: 0.6791 - val_loss: 0.5879 - val_acc: 0.6844
Epoch 3/100
6180/6180 [==============================] - 1s 150us/step - loss: 0.5833 - acc: 0.6791 - val_loss: 0.5727 - val_acc: 0.6844
Epoch 4/100
6180/6180 [==============================] - 1s 170us/step - loss: 0.5664 - acc: 0.6791 - val_loss: 0.5542 - val_acc: 0.6844
Epoch 5/100
6180/6180 [==============================] - 1s 174us/step - loss: 0.5470 - acc: 0.6833 - val_loss: 0.5341 - val_acc: 0.7063
Epoch 6/100
6180/6180 [==============================] - 1s 157us/step - loss: 0.5252 - acc: 0.7170 - val_loss: 0.5118 - val_acc: 0.7373
Epoch 7/100
6180/6180 [==============================] - 1s 161us/step - loss: 0.5017 - acc: 0.7406 - val_loss: 0.4899 - val_acc:

6180/6180 [==============================] - 1s 158us/step - loss: 0.3326 - acc: 0.8510 - val_loss: 0.3451 - val_acc: 0.8437
Epoch 59/100
6180/6180 [==============================] - 1s 156us/step - loss: 0.3322 - acc: 0.8511 - val_loss: 0.3468 - val_acc: 0.8407
Epoch 60/100
6180/6180 [==============================] - 1s 166us/step - loss: 0.3325 - acc: 0.8497 - val_loss: 0.3444 - val_acc: 0.8445
Epoch 61/100
6180/6180 [==============================] - 1s 156us/step - loss: 0.3321 - acc: 0.8490 - val_loss: 0.3443 - val_acc: 0.8445
Epoch 62/100
6180/6180 [==============================] - 1s 156us/step - loss: 0.3320 - acc: 0.8500 - val_loss: 0.3449 - val_acc: 0.8422
Epoch 63/100
6180/6180 [==============================] - 1s 156us/step - loss: 0.3320 - acc: 0.8510 - val_loss: 0.3441 - val_acc: 0.8445
Epoch 64/100
6180/6180 [==============================] - 1s 172us/step - loss: 0.3318 - acc: 0.8508 - val_loss: 0.3450 - val_acc: 0.8422
Epoch 65/100
6180/6180 [=======================

In [20]:
pala = []
for i in range(0,len(sentence_vectors)):
    pala.append(np.count_nonzero(sentence_vectors[i]))
print(max(pala))

4110

Reading parcial data

In [259]:
#retains = [0.0100000000000000,0.0600000000000000,0.110000000000000,0.160000000000000,0.210000000000000,0.260000000000000,0.310000000000000,0.360000000000000,0.410000000000000,0.460000000000000,0.510000000000000,0.560000000000000,0.610000000000000,0.660000000000000,0.710000000000000,0.760000000000000,0.810000000000000,0.860000000000000,0.910000000000000,1]
retains = [0.910000000000000]

for j in range (0,len(retains)):
    rXtest = np.zeros((X_test.shape[0], num_terms))
    for i in range(0,X_test.shape[0]): # iterating each test document
        noz = np.where(S_test[i] != 0)[0] # from our idx (ONLY INDXS) document matriz, extract idx != 0 (this step could be avoid if we do not agregate those 0)
        ntermssf = round(len(noz)*retains[j]) # We obtain the number of terns with x% of longitud
        myox = 1 # minnum Termos = 1
        wdix = 0 # 
        freqtsof = np.zeros(num_terms)  
        # This vector activate the terms present in the Document until that longitud
        while myox <= ntermssf:
            if S_test[i,wdix] != 0:
                freqtsof[int(S_test[i,wdix])] = 1 #FIX +1 ????!!
                myox += 1
            wdix += 1
        rXtest[i] = freqtsof
    #Xtestv = rXtest

In [ ]:
hacer funcion las representaciones
verificar el pre procesamietno
PBR: u did this for all X, u shoul do this 4 xtrain and x test

